# X-Ray Diffraction

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import scipy.optimize as opt
import scipy.constants as const

title_size = 20
axis_size = 16

## Pre-lab Questions

1.a) Read labscript and relevant course material. Elements used from mathematical techniques are: scipy curve_fit and chi squared test.
<br>
<br>
1.b) From the equation $m\lambda = 2d\sin\theta$ from this for a higher wavelength the angle of diffraction will be greater than for a lower wavelength. As the energy of the K-alpha is lower as it comes from the n = 2 to the n = 1 shell, the wavelength will be higher and the angle of diffraction will be greater then that of the K-beta which comes from the n = 3 to the n = 1 shell.
<br>
<br>
1.c) Calculate the Interatomic spacing of a LiF cubic crystal using $d = \frac{a}{\sqrt{3}}$ where a is the lattice constant. First calculate the lattice constant from the mass and the density of the crystal. 
<br>
<br>
$\rho = 2.64 g/cm^3$
<br>
<br>
$m_{tot} = 25.94 u$
<br>
<br>
$m_{tot} = \frac{25.94}{6.022\times 10^{23}} = 4.308\times 10^{-23} g$
<br>
<br>
$Volume = \frac{m_{tot}}{\rho} = \frac{4.308\times 10^{-23}}{2.64} = 1.632\times 10^{-23} cm^3$
<br>
<br>
$a = \sqrt[3]{Volume} = \sqrt[3]{1.632\times 10^{-23}} = 2.536\times 10^{-8} cm$
<br>
<br>
$d = \frac{a}{\sqrt{3}} = \frac{2.536\times 10^{-8}}{\sqrt{3}} = 1.46\times 10^{-8} cm$